In [12]:
from datetime import timedelta
import re
from collections import defaultdict

def parse_time_str(time_str):
    h, m, s = time_str.strip().split(':')

    # Handle microseconds if present
    if '.' in s:
        sec, micro = s.split('.')
        micro = int(micro.ljust(6, '0'))  # pad to microseconds if needed
    else:
        sec = s
        micro = 0

    return timedelta(hours=int(h), minutes=int(m), seconds=int(sec), microseconds=micro)

def calculate_time_stats_from_text(data):
    cluster_times = defaultdict(list)
    lines = data.strip().split('\n')

    for line in lines:
        match = re.search(r'Cluster Size: (\d+), Time: (\d+:\d+:\d+\.\d+)', line)
        if match:
            size = int(match.group(1))
            time_str = match.group(2)
            duration = parse_time_str(time_str)  # Ensure this function returns timedelta
            cluster_times[size].append(duration)

    stats_in_minutes = {}
    for size, times in cluster_times.items():
        total = sum(times, timedelta())
        avg = total / len(times)
        min_time = min(times)
        max_time = max(times)
        
        stats_in_minutes[size] = {
            'average_minutes': avg.total_seconds() / 60,
            'min_minutes': min_time.total_seconds() / 60,
            'max_minutes': max_time.total_seconds() / 60
        }

    return stats_in_minutes

def get_latest_time_per_flight(text_block):
    flight_latest = {}

    for line in text_block.strip().split('\n'):
        time_match = re.search(r'Time: (\d+:\d+:\d+\.\d+)', line)
        flight_match = re.search(r'Flight number: (\w+\d+)', line)

        if time_match and flight_match:
            time = parse_time_str(time_match.group(1))
            flight = flight_match.group(1)

            if flight not in flight_latest or time > flight_latest[flight]:
                flight_latest[flight] = time

    return flight_latest


def calculate_overall_time_stats(data):
    all_times = []

    lines = data.strip().split('\n')
    for line in lines:
        match = re.search(r'Cluster Size: (\d+), Time: (\d+:\d+:\d+\.\d+)', line)
        if match:
            time_str = match.group(2)
            duration = parse_time_str(time_str)
            all_times.append(duration)

    if not all_times:
        return {"average_minutes": None, "min_minutes": None, "max_minutes": None}

    total = sum(all_times, timedelta())
    avg = total / len(all_times)
    min_time = min(all_times)
    max_time = max(all_times)

    return {
        "average_minutes": avg.total_seconds() / 60,
        "min_minutes": min_time.total_seconds() / 60,
        "max_minutes": max_time.total_seconds() / 60
    }

'''
def get_latest_time_per_flight(data):
    latest_times = {}

    for line in data.strip().splitlines():
        match = re.search(r'Time: ([\d:.]+).*Flight number: (\d+)', line)
        if match:
            time_str, flight_num = match.groups()
            td = parse_time_str(time_str)
            if flight_num not in latest_times or td > latest_times[flight_num]:
                latest_times[flight_num] = td

    return latest_times
   ''' 
def timedelta_to_minutes(td):
    return td.total_seconds() / 60
# Example usage with direct text input:
data = """
Reservation ID: RES-00016, Cluster Size: 2, Time: 0:02:25.180000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00041, Cluster Size: 6, Time: 0:04:06.460000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00009, Cluster Size: 2, Time: 0:04:28.340000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00241, Cluster Size: 3, Time: 0:04:44.140000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00019, Cluster Size: 3, Time: 0:05:02.020000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00053, Cluster Size: 7, Time: 0:05:04.300000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00302, Cluster Size: 7, Time: 0:05:08.260000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00256, Cluster Size: 5, Time: 0:05:26.200000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00066, Cluster Size: 4, Time: 0:05:35.360000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00386, Cluster Size: 7, Time: 0:05:50.620000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00003, Cluster Size: 9, Time: 0:05:50.940000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00357, Cluster Size: 5, Time: 0:05:50.940000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00045, Cluster Size: 5, Time: 0:05:52.560000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00351, Cluster Size: 7, Time: 0:05:52.880000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00001, Cluster Size: 5, Time: 0:06:13.360000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00358, Cluster Size: 3, Time: 0:06:19.800000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00043, Cluster Size: 7, Time: 0:06:19.800000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00413, Cluster Size: 3, Time: 0:06:38.440000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00007, Cluster Size: 8, Time: 0:06:44.540000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00057, Cluster Size: 6, Time: 0:06:49.960000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00013, Cluster Size: 3, Time: 0:06:50.280000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00039, Cluster Size: 6, Time: 0:06:57.920000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00415, Cluster Size: 4, Time: 0:07:00.560000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00372, Cluster Size: 5, Time: 0:07:05.480000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00253, Cluster Size: 8, Time: 0:07:08.740000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00248, Cluster Size: 6, Time: 0:07:19.560000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00277, Cluster Size: 3, Time: 0:07:40.060000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00067, Cluster Size: 4, Time: 0:07:54.140000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00010, Cluster Size: 6, Time: 0:07:54.460000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00044, Cluster Size: 2, Time: 0:08:18.540000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00368, Cluster Size: 2, Time: 0:08:29.700000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00367, Cluster Size: 3, Time: 0:08:40.700000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00362, Cluster Size: 2, Time: 0:08:40.700000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00029, Cluster Size: 7, Time: 0:08:46.580000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00259, Cluster Size: 5, Time: 0:08:48.160000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00428, Cluster Size: 3, Time: 0:08:52.640000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00006, Cluster Size: 9, Time: 0:08:53.620000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00026, Cluster Size: 10, Time: 0:08:53.620000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00364, Cluster Size: 2, Time: 0:08:54.620000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00072, Cluster Size: 5, Time: 0:09:12.720000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00018, Cluster Size: 5, Time: 0:09:17.680000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00056, Cluster Size: 8, Time: 0:09:22.600000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00242, Cluster Size: 3, Time: 0:09:33.900000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00435, Cluster Size: 3, Time: 0:10:09.280000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00261, Cluster Size: 7, Time: 0:10:21.340000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00239, Cluster Size: 3, Time: 0:10:24.960000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00323, Cluster Size: 5, Time: 0:10:28.200000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00273, Cluster Size: 2, Time: 0:10:39.100000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00390, Cluster Size: 3, Time: 0:10:41.680000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00237, Cluster Size: 7, Time: 0:10:42.320000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00284, Cluster Size: 3, Time: 0:10:43, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00069, Cluster Size: 4, Time: 0:10:44.300000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00289, Cluster Size: 6, Time: 0:10:48.900000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00278, Cluster Size: 7, Time: 0:10:54.760000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00015, Cluster Size: 6, Time: 0:11:26.160000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00055, Cluster Size: 2, Time: 0:11:26.500000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00403, Cluster Size: 4, Time: 0:11:49, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00401, Cluster Size: 5, Time: 0:11:50, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00325, Cluster Size: 7, Time: 0:11:50, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00002, Cluster Size: 6, Time: 0:12:04.860000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00412, Cluster Size: 5, Time: 0:12:06.500000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00269, Cluster Size: 3, Time: 0:12:18, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00060, Cluster Size: 6, Time: 0:12:21.240000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00046, Cluster Size: 3, Time: 0:12:25.220000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00295, Cluster Size: 6, Time: 0:12:33.440000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00352, Cluster Size: 5, Time: 0:13:26.140000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00023, Cluster Size: 6, Time: 0:13:48.580000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00054, Cluster Size: 6, Time: 0:13:49.580000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00058, Cluster Size: 3, Time: 0:13:49.580000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00270, Cluster Size: 2, Time: 0:13:51.280000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00065, Cluster Size: 5, Time: 0:13:54.900000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00020, Cluster Size: 5, Time: 0:13:58.640000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00004, Cluster Size: 7, Time: 0:14:05.600000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00028, Cluster Size: 6, Time: 0:14:07.900000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00293, Cluster Size: 4, Time: 0:14:10.600000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00048, Cluster Size: 6, Time: 0:14:24.060000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00280, Cluster Size: 2, Time: 0:15:02.800000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00052, Cluster Size: 7, Time: 0:15:21.100000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00262, Cluster Size: 7, Time: 0:15:39.420000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00075, Cluster Size: 5, Time: 0:16:07.240000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00037, Cluster Size: 6, Time: 0:16:13.500000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00392, Cluster Size: 2, Time: 0:16:16.480000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00145, Cluster Size: 4, Time: 0:03:35.200000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00035, Cluster Size: 3, Time: 0:16:41.720000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00071, Cluster Size: 3, Time: 0:16:53.100000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00330, Cluster Size: 9, Time: 0:17:14.180000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00333, Cluster Size: 7, Time: 0:04:18.160000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00146, Cluster Size: 9, Time: 0:04:18.480000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00257, Cluster Size: 4, Time: 0:17:19.460000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00350, Cluster Size: 5, Time: 0:17:21.100000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00275, Cluster Size: 4, Time: 0:17:28.220000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00434, Cluster Size: 2, Time: 0:04:57.260000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00152, Cluster Size: 7, Time: 0:05:07.980000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00134, Cluster Size: 6, Time: 0:05:09.280000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00104, Cluster Size: 6, Time: 0:05:12.840000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00359, Cluster Size: 7, Time: 0:05:24.240000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00319, Cluster Size: 2, Time: 0:05:26.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00141, Cluster Size: 3, Time: 0:05:39.240000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00064, Cluster Size: 2, Time: 0:18:44.480000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00304, Cluster Size: 8, Time: 0:18:47.400000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00410, Cluster Size: 5, Time: 0:05:53.260000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00040, Cluster Size: 6, Time: 0:18:58.540000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00024, Cluster Size: 4, Time: 0:19:00.800000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00030, Cluster Size: 4, Time: 0:19:02.320000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00086, Cluster Size: 5, Time: 0:06:11.640000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00061, Cluster Size: 2, Time: 0:19:14.260000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00340, Cluster Size: 2, Time: 0:19:15.240000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00353, Cluster Size: 2, Time: 0:19:53.740000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00324, Cluster Size: 2, Time: 0:19:57.360000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00091, Cluster Size: 2, Time: 0:07:29.260000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00084, Cluster Size: 9, Time: 0:07:29.580000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00349, Cluster Size: 3, Time: 0:20:35.800000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00236, Cluster Size: 3, Time: 0:07:39.780000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00433, Cluster Size: 5, Time: 0:07:40.420000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00406, Cluster Size: 4, Time: 0:20:48.600000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00306, Cluster Size: 6, Time: 0:07:54.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00128, Cluster Size: 4, Time: 0:07:56.460000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00073, Cluster Size: 9, Time: 0:20:56.800000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00122, Cluster Size: 4, Time: 0:08:02.080000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00161, Cluster Size: 9, Time: 0:08:31.920000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00034, Cluster Size: 3, Time: 0:21:50.260000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00062, Cluster Size: 3, Time: 0:21:51.560000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00282, Cluster Size: 3, Time: 0:21:51.880000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00008, Cluster Size: 3, Time: 0:21:52.540000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00157, Cluster Size: 5, Time: 0:08:53.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00032, Cluster Size: 4, Time: 0:21:57.180000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00332, Cluster Size: 4, Time: 0:22:03.100000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00111, Cluster Size: 2, Time: 0:09:21.620000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00303, Cluster Size: 2, Time: 0:22:26.580000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00240, Cluster Size: 2, Time: 0:09:29.800000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00148, Cluster Size: 3, Time: 0:09:34.340000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00371, Cluster Size: 3, Time: 0:09:37.620000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00076, Cluster Size: 4, Time: 0:22:51.940000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00301, Cluster Size: 4, Time: 0:23:15.420000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00074, Cluster Size: 3, Time: 0:23:15.760000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00090, Cluster Size: 7, Time: 0:10:17.360000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00117, Cluster Size: 5, Time: 0:10:20.960000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00012, Cluster Size: 6, Time: 0:23:22.300000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00366, Cluster Size: 6, Time: 0:23:34.260000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00011, Cluster Size: 6, Time: 0:24:10.460000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00120, Cluster Size: 7, Time: 0:11:31.480000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00109, Cluster Size: 6, Time: 0:11:46.780000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00051, Cluster Size: 7, Time: 0:24:53.300000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00036, Cluster Size: 5, Time: 0:24:57.180000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00321, Cluster Size: 5, Time: 0:12:07.520000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00375, Cluster Size: 4, Time: 0:12:17.580000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00017, Cluster Size: 4, Time: 0:25:22.920000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00431, Cluster Size: 3, Time: 0:12:55.380000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00287, Cluster Size: 2, Time: 0:13:06.180000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00022, Cluster Size: 2, Time: 0:26:09.580000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00014, Cluster Size: 7, Time: 0:26:38.900000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00106, Cluster Size: 7, Time: 0:13:50.040000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00077, Cluster Size: 4, Time: 0:26:51.380000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00131, Cluster Size: 7, Time: 0:14:01.160000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00049, Cluster Size: 6, Time: 0:27:07.560000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00144, Cluster Size: 4, Time: 0:14:09.180000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00243, Cluster Size: 4, Time: 0:27:10.140000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00398, Cluster Size: 10, Time: 0:14:18.600000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00021, Cluster Size: 2, Time: 0:27:42.280000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00384, Cluster Size: 5, Time: 0:14:56.760000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00427, Cluster Size: 7, Time: 0:15:27.060000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00286, Cluster Size: 4, Time: 0:28:27.720000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00098, Cluster Size: 7, Time: 0:15:47.780000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00101, Cluster Size: 7, Time: 0:15:54.400000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00059, Cluster Size: 6, Time: 0:29:00, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00159, Cluster Size: 4, Time: 0:16:12.780000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00038, Cluster Size: 7, Time: 0:29:14.860000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00272, Cluster Size: 3, Time: 0:29:20.960000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00399, Cluster Size: 5, Time: 0:29:50.120000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00158, Cluster Size: 4, Time: 0:16:52.880000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00150, Cluster Size: 4, Time: 0:16:56.600000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00027, Cluster Size: 2, Time: 0:30:02.120000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00294, Cluster Size: 4, Time: 0:04:17.180000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00328, Cluster Size: 3, Time: 0:17:21.360000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00087, Cluster Size: 4, Time: 0:17:27.920000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00224, Cluster Size: 8, Time: 0:04:56.580000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00310, Cluster Size: 6, Time: 0:04:59.460000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00309, Cluster Size: 4, Time: 0:05:00.580000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00025, Cluster Size: 3, Time: 0:31:35.020000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00099, Cluster Size: 3, Time: 0:18:47.740000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00118, Cluster Size: 4, Time: 0:18:52.100000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00374, Cluster Size: 5, Time: 0:19:02.620000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00376, Cluster Size: 8, Time: 0:19:17.100000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00135, Cluster Size: 5, Time: 0:19:17.440000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00263, Cluster Size: 6, Time: 0:06:17.440000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00409, Cluster Size: 2, Time: 0:32:18.100000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00336, Cluster Size: 2, Time: 0:32:21.640000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00290, Cluster Size: 6, Time: 0:32:29.520000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00184, Cluster Size: 5, Time: 0:06:43.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00326, Cluster Size: 5, Time: 0:32:57.120000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00341, Cluster Size: 6, Time: 0:06:57.120000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00299, Cluster Size: 5, Time: 0:06:58.100000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00225, Cluster Size: 4, Time: 0:07:03.260000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00113, Cluster Size: 6, Time: 0:20:08.820000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00318, Cluster Size: 7, Time: 0:20:34.440000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00266, Cluster Size: 4, Time: 0:20:46.540000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00126, Cluster Size: 3, Time: 0:20:47.200000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00171, Cluster Size: 2, Time: 0:07:51.320000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00142, Cluster Size: 3, Time: 0:21:08.720000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00204, Cluster Size: 8, Time: 0:08:24.420000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00400, Cluster Size: 3, Time: 0:08:27.300000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00176, Cluster Size: 4, Time: 0:08:47.140000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00387, Cluster Size: 3, Time: 0:21:56.260000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00414, Cluster Size: 2, Time: 0:22:02.920000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00271, Cluster Size: 7, Time: 0:09:19.140000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00130, Cluster Size: 7, Time: 0:22:19.460000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00047, Cluster Size: 3, Time: 0:35:24.280000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00147, Cluster Size: 7, Time: 0:22:27.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00378, Cluster Size: 5, Time: 0:22:32.080000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00068, Cluster Size: 5, Time: 0:35:33.780000, Flight time: 10:34, Flight number: EK4179
Reservation ID: RES-00424, Cluster Size: 2, Time: 0:22:42.560000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00129, Cluster Size: 5, Time: 0:22:52.680000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00137, Cluster Size: 6, Time: 0:22:52.680000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00112, Cluster Size: 3, Time: 0:22:53.320000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00153, Cluster Size: 10, Time: 0:23:04.780000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00167, Cluster Size: 7, Time: 0:10:26.200000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00370, Cluster Size: 5, Time: 0:10:29.440000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00100, Cluster Size: 2, Time: 0:23:32, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00107, Cluster Size: 2, Time: 0:23:32.320000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00419, Cluster Size: 6, Time: 0:10:32.960000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00214, Cluster Size: 5, Time: 0:10:37.180000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00314, Cluster Size: 3, Time: 0:10:40.460000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00092, Cluster Size: 2, Time: 0:23:42.060000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00175, Cluster Size: 5, Time: 0:11:43.320000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00437, Cluster Size: 6, Time: 0:24:54.160000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00213, Cluster Size: 5, Time: 0:11:58.860000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00206, Cluster Size: 2, Time: 0:12:05.420000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00291, Cluster Size: 5, Time: 0:25:13.140000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00094, Cluster Size: 4, Time: 0:25:22.380000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00385, Cluster Size: 6, Time: 0:25:22.720000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00183, Cluster Size: 6, Time: 0:12:22.720000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00172, Cluster Size: 10, Time: 0:12:27.220000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00252, Cluster Size: 10, Time: 0:12:36.820000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00345, Cluster Size: 10, Time: 0:25:44.480000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00102, Cluster Size: 10, Time: 0:26:03.540000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00124, Cluster Size: 3, Time: 0:26:07.520000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00096, Cluster Size: 6, Time: 0:26:08.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00245, Cluster Size: 4, Time: 0:26:10.240000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00215, Cluster Size: 6, Time: 0:13:48.540000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00140, Cluster Size: 5, Time: 0:26:54.740000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00218, Cluster Size: 9, Time: 0:14:05.200000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00296, Cluster Size: 6, Time: 0:27:18.340000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00281, Cluster Size: 6, Time: 0:27:19.640000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00180, Cluster Size: 5, Time: 0:14:24.900000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00421, Cluster Size: 2, Time: 0:27:34.160000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00265, Cluster Size: 7, Time: 0:14:54.540000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00081, Cluster Size: 6, Time: 0:28:05.320000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00178, Cluster Size: 6, Time: 0:15:09.520000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00103, Cluster Size: 5, Time: 0:28:11.440000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00149, Cluster Size: 3, Time: 0:28:14.660000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00235, Cluster Size: 6, Time: 0:15:23.900000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00139, Cluster Size: 7, Time: 0:28:31.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00432, Cluster Size: 3, Time: 0:28:51.880000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00363, Cluster Size: 2, Time: 0:28:53.200000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00080, Cluster Size: 7, Time: 0:28:57.880000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00164, Cluster Size: 2, Time: 0:16:04.080000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00369, Cluster Size: 5, Time: 0:29:08.700000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00228, Cluster Size: 5, Time: 0:17:09.520000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00316, Cluster Size: 2, Time: 0:30:14.100000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00182, Cluster Size: 4, Time: 0:17:23.560000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00089, Cluster Size: 3, Time: 0:30:36.620000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00116, Cluster Size: 2, Time: 0:30:36.960000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00250, Cluster Size: 5, Time: 0:17:37.960000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00082, Cluster Size: 3, Time: 0:30:40.240000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00169, Cluster Size: 10, Time: 0:17:47.060000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00274, Cluster Size: 3, Time: 0:17:47.060000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00268, Cluster Size: 4, Time: 0:17:56.200000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00226, Cluster Size: 4, Time: 0:18:06, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00391, Cluster Size: 3, Time: 0:31:09.860000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00230, Cluster Size: 7, Time: 0:18:30.580000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00348, Cluster Size: 5, Time: 0:32:09.820000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00317, Cluster Size: 3, Time: 0:32:37.560000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00422, Cluster Size: 7, Time: 0:19:40.460000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00418, Cluster Size: 6, Time: 0:19:50.220000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00339, Cluster Size: 8, Time: 0:20:28.160000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00234, Cluster Size: 4, Time: 0:20:34.380000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00425, Cluster Size: 4, Time: 0:33:40.580000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00078, Cluster Size: 4, Time: 0:33:41.860000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00305, Cluster Size: 4, Time: 0:33:45.160000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00360, Cluster Size: 7, Time: 0:34:05.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00300, Cluster Size: 6, Time: 0:21:25.040000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00115, Cluster Size: 3, Time: 0:35:06.800000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00192, Cluster Size: 7, Time: 0:22:30.880000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00429, Cluster Size: 3, Time: 0:22:31.900000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00247, Cluster Size: 4, Time: 0:36:08.580000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00308, Cluster Size: 7, Time: 0:23:29.140000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00121, Cluster Size: 2, Time: 0:36:48.640000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00155, Cluster Size: 4, Time: 0:36:48.640000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00085, Cluster Size: 2, Time: 0:36:49.600000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00311, Cluster Size: 4, Time: 0:24:33.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00229, Cluster Size: 5, Time: 0:24:36.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00095, Cluster Size: 5, Time: 0:37:50.580000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00377, Cluster Size: 5, Time: 0:25:10.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00114, Cluster Size: 5, Time: 0:38:11.880000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00168, Cluster Size: 3, Time: 0:25:16.560000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00123, Cluster Size: 3, Time: 0:39:07.120000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00354, Cluster Size: 4, Time: 0:39:29.320000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00285, Cluster Size: 6, Time: 0:26:35.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00331, Cluster Size: 2, Time: 0:39:52.880000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00246, Cluster Size: 7, Time: 0:39:55.880000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00079, Cluster Size: 7, Time: 0:39:56.840000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00202, Cluster Size: 2, Time: 0:26:56.840000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00160, Cluster Size: 3, Time: 0:40:07.400000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00365, Cluster Size: 4, Time: 0:27:09.900000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00197, Cluster Size: 2, Time: 0:27:15.920000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00210, Cluster Size: 2, Time: 0:27:20.140000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00165, Cluster Size: 7, Time: 0:28:35.200000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00288, Cluster Size: 3, Time: 0:28:39.800000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00355, Cluster Size: 7, Time: 0:41:51.260000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00279, Cluster Size: 2, Time: 0:41:59.940000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00356, Cluster Size: 5, Time: 0:29:15.820000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00337, Cluster Size: 5, Time: 0:42:33.200000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00151, Cluster Size: 2, Time: 0:42:50.240000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00388, Cluster Size: 3, Time: 0:42:58.780000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00417, Cluster Size: 3, Time: 0:43:03.240000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00405, Cluster Size: 3, Time: 0:30:03.580000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00174, Cluster Size: 2, Time: 0:30:04.200000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00097, Cluster Size: 2, Time: 0:43:18.320000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00344, Cluster Size: 2, Time: 0:44:10.640000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00154, Cluster Size: 6, Time: 0:44:18.100000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00093, Cluster Size: 4, Time: 0:44:48.860000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00156, Cluster Size: 3, Time: 0:45:16.220000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00110, Cluster Size: 2, Time: 0:45:32.480000, Flight time: 10:47, Flight number: BA6532
Reservation ID: RES-00205, Cluster Size: 4, Time: 0:33:34.280000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00221, Cluster Size: 6, Time: 0:33:37.180000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00383, Cluster Size: 5, Time: 0:33:47.080000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00232, Cluster Size: 7, Time: 0:34:12.080000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00430, Cluster Size: 6, Time: 0:35:03.500000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00209, Cluster Size: 3, Time: 0:35:16.300000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00346, Cluster Size: 5, Time: 0:36:30.320000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00217, Cluster Size: 2, Time: 0:36:45.160000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00198, Cluster Size: 4, Time: 0:37:06.760000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00393, Cluster Size: 7, Time: 0:37:17.360000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00238, Cluster Size: 4, Time: 0:37:17.700000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00254, Cluster Size: 4, Time: 0:37:20.680000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00189, Cluster Size: 7, Time: 0:39:36.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00244, Cluster Size: 7, Time: 0:39:47.500000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00251, Cluster Size: 4, Time: 0:39:50.500000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00264, Cluster Size: 9, Time: 0:40:38.860000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00298, Cluster Size: 7, Time: 0:41:06.200000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00220, Cluster Size: 3, Time: 0:41:06.840000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00200, Cluster Size: 5, Time: 0:42:40.580000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00408, Cluster Size: 3, Time: 0:42:41.880000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00342, Cluster Size: 7, Time: 0:42:42.240000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00195, Cluster Size: 9, Time: 0:43:48.020000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00177, Cluster Size: 7, Time: 0:43:52.340000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00394, Cluster Size: 5, Time: 0:44:54.560000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00338, Cluster Size: 7, Time: 0:45:46.900000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00208, Cluster Size: 7, Time: 0:45:51.120000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00402, Cluster Size: 4, Time: 0:45:51.800000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00201, Cluster Size: 5, Time: 0:47:02.840000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00216, Cluster Size: 5, Time: 0:47:29.360000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00179, Cluster Size: 10, Time: 0:47:54.700000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00211, Cluster Size: 4, Time: 0:48:53.500000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00203, Cluster Size: 6, Time: 0:48:58.440000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00166, Cluster Size: 7, Time: 0:48:59.740000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00190, Cluster Size: 4, Time: 0:49:43.320000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00196, Cluster Size: 5, Time: 0:50:04.420000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00379, Cluster Size: 5, Time: 0:51:26.780000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00255, Cluster Size: 4, Time: 0:51:35.360000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00436, Cluster Size: 6, Time: 0:52:03.860000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00380, Cluster Size: 7, Time: 0:52:07.840000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00181, Cluster Size: 6, Time: 0:52:55.380000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00426, Cluster Size: 5, Time: 0:52:58.380000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00223, Cluster Size: 5, Time: 0:53:36.140000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00173, Cluster Size: 7, Time: 0:53:49.280000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00334, Cluster Size: 6, Time: 0:54:53.100000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00231, Cluster Size: 5, Time: 0:55:00.040000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00193, Cluster Size: 4, Time: 0:55:23.220000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00312, Cluster Size: 2, Time: 0:55:38.140000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00207, Cluster Size: 7, Time: 0:55:55.360000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00373, Cluster Size: 5, Time: 0:56:35.400000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00188, Cluster Size: 4, Time: 0:57:36.880000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00283, Cluster Size: 7, Time: 0:58:22.720000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00260, Cluster Size: 6, Time: 0:58:34.660000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00233, Cluster Size: 3, Time: 0:58:47.620000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00222, Cluster Size: 3, Time: 0:58:48.940000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00199, Cluster Size: 3, Time: 0:59:11.760000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00185, Cluster Size: 3, Time: 1:00:44.360000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00227, Cluster Size: 5, Time: 1:01:00.160000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00212, Cluster Size: 3, Time: 1:01:01.520000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00343, Cluster Size: 4, Time: 1:01:21.900000, Flight time: 11:00, Flight number: SV5592
Reservation ID: RES-00329, Cluster Size: 2, Time: 1:01:50.060000, Flight time: 11:00, Flight number: SV5592
"""

stats = calculate_time_stats_from_text(data)
for cluster_size, values in sorted(stats.items()):
    print(f"Cluster Size {cluster_size}:")
    print(f"  Average CDT: {values['average_minutes']:.3f} minutes")
    print(f"  Min CDT:     {values['min_minutes']:.3f} minutes")
    print(f"  Max CDT:     {values['max_minutes']:.3f} minutes")
    print()

stats2 = calculate_overall_time_stats(data)
print(f"Average PDT: {stats2['average_minutes']:.3f} minutes")
print(f"Min PDT:     {stats2['min_minutes']:.3f} minutes")
print(f"Max PDT:     {stats2['max_minutes']:.3f} minutes")
print()

latest_times = get_latest_time_per_flight(data)
for flight, td in latest_times.items():
    print(f"{flight} FDT: {timedelta_to_minutes(td):.3f} minutes")

Cluster Size 2:
  Average CDT: 23.500 minutes
  Min CDT:     2.420 minutes
  Max CDT:     61.834 minutes

Cluster Size 3:
  Average CDT: 24.477 minutes
  Min CDT:     4.736 minutes
  Max CDT:     61.025 minutes

Cluster Size 4:
  Average CDT: 25.229 minutes
  Min CDT:     3.587 minutes
  Max CDT:     61.365 minutes

Cluster Size 5:
  Average CDT: 23.942 minutes
  Min CDT:     5.437 minutes
  Max CDT:     61.003 minutes

Cluster Size 6:
  Average CDT: 20.778 minutes
  Min CDT:     4.108 minutes
  Max CDT:     58.578 minutes

Cluster Size 7:
  Average CDT: 24.502 minutes
  Min CDT:     4.303 minutes
  Max CDT:     58.379 minutes

Cluster Size 8:
  Average CDT: 11.895 minutes
  Min CDT:     4.943 minutes
  Max CDT:     20.469 minutes

Cluster Size 9:
  Average CDT: 17.179 minutes
  Min CDT:     4.308 minutes
  Max CDT:     43.800 minutes

Cluster Size 10:
  Average CDT: 20.983 minutes
  Min CDT:     8.894 minutes
  Max CDT:     47.912 minutes

Average PDT: 23.309 minutes
Min PDT:     2.42

<>:87: SyntaxWarning: invalid escape sequence '\d'
<>:87: SyntaxWarning: invalid escape sequence '\d'
/var/folders/1q/vly_xpw52ss__8lr1wl63j2h0000gn/T/ipykernel_23847/541532287.py:87: SyntaxWarning: invalid escape sequence '\d'
  '''
